# 程序说明
时间：2016年11月16日

说明：该程序是一个包含两个卷积层、一个池化层和一个全连接层组成的神经网络。

数据集：MNIST

## 1.加载keras模块

In [1]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

In [2]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


#### 如需绘制模型请加载plot

In [3]:
from keras.utils.visualize_util import plot

## 2.变量初始化

In [4]:
batch_size = 128 
nb_classes = 10
nb_epoch = 12

# 输入数据的维度
img_rows, img_cols = 28, 28
# 使用的卷积滤波器的数量
nb_filters = 32
# 用于 max pooling 的池化面积
pool_size = (2, 2)
# 卷积核的尺寸
kernel_size = (3, 3)

## 3.准备数据
需要将数据还原成28*28大小的数组

注意：Theano和TensorFlow的维度表示含义是不一样的。

'th'模式，也即Theano模式会把100张RGB三通道的16×32（高为16宽为32）彩色图表示为下面这种形式（100,3,16,32），Caffe采取的也是这种方式。第0个维度是样本维，代表样本的数目，第1个维度是通道维，代表颜色通道数。后面两个就是高和宽了。而TensorFlow，即'tf'模式的表达形式是（100,16,32,3），即把通道维放在了最后。这两个表达方法本质上没有什么区别。

In [5]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()

if K.image_dim_ordering() == 'th':
    X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
    X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
    X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


### 转换类标号

In [6]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

## 4.建立模型
### 使用Sequential（）

In [7]:
model = Sequential()

model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                        border_mode='valid',
                        input_shape=input_shape))
model.add(Activation('relu'))
model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

### 打印模型

In [8]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 26, 26, 32)    320         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 26, 26, 32)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 24, 24, 32)    9248        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 24, 24, 32)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________

### 绘制模型结构图，并保存成图片

In [9]:
plot(model, to_file='model-cnn.png')

#### 显示绘制的图片
![image](http://p1.bpimg.com/4851/4c386bd0784d9b4d.png)

## 5.训练与评估
### 编译模型

In [10]:
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

### 迭代训练

In [11]:
model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch,
          verbose=1, validation_data=(X_test, Y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 13s - loss: 0.3825 - acc: 0.8821 - val_loss: 0.0861 - val_acc: 0.9734
Epoch 2/12
60000/60000 [==============================] - 12s - loss: 0.1333 - acc: 0.9609 - val_loss: 0.0611 - val_acc: 0.9801
Epoch 3/12
60000/60000 [==============================] - 12s - loss: 0.1041 - acc: 0.9694 - val_loss: 0.0545 - val_acc: 0.9824
Epoch 4/12
60000/60000 [==============================] - 12s - loss: 0.0861 - acc: 0.9736 - val_loss: 0.0441 - val_acc: 0.9852
Epoch 5/12
60000/60000 [==============================] - 12s - loss: 0.0780 - acc: 0.9767 - val_loss: 0.0410 - val_acc: 0.9862
Epoch 6/12
60000/60000 [==============================] - 12s - loss: 0.0701 - acc: 0.9795 - val_loss: 0.0384 - val_acc: 0.9869
Epoch 7/12
60000/60000 [==============================] - 12s - loss: 0.0624 - acc: 0.9815 - val_loss: 0.0378 - val_acc: 0.9873
Epoch 8/12
60000/60000 [==============================

### 模型评估

In [12]:
score = model.evaluate(X_test, Y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Test score: 0.0321088282784
Test accuracy: 0.9896
